In [ ]:
import tensorflow as tf

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split


In [ ]:
dt=pd.read_csv("/content/csv_result-bone-marrow.csv")

In [ ]:
dt.head()
#display the first few rows of a pandas DataFrame

,id,Recipientgender,Stemcellsource,Donorage,Donorage35,IIIV,Gendermatch,DonorABO,RecipientABO,RecipientRh,...,extcGvHD,CD34kgx10d6,CD3dCD34,CD3dkgx10d8,Rbodymass,ANCrecovery,PLTrecovery,time_to_aGvHD_III_IV,survival_time,survival_status
0,1,1,1,22.830137,0,1,0,1,1,1,...,1,7.20,1.33876,5.38,35,19,51,32,999,0
1,2,1,0,23.342466,0,1,0,-1,-1,1,...,1,4.50,11.078295,0.41,20.6,16,37,1000000,163,1
2,3,1,0,26.394521,0,1,0,-1,-1,1,...,1,7.94,19.01323,0.42,23.4,23,20,1000000,435,1
3,4,0,0,39.684932,1,1,0,1,2,1,...,?,4.25,29.481647,0.14,50,23,29,19,53,1
4,5,0,1,33.358904,0,0,0,1,2,0,...,1,51.85,3.972255,13.05,9,14,14,1000000,2043,0


In [ ]:
dt=dt.drop(['id'], axis=1)
#remove a column named 'id' from the DataFrame dt

In [ ]:
dt.shape
#trieve the number of rows and columns

(187, 37)

In [ ]:
dt.dtypes

Recipientgender           int64
Stemcellsource            int64
Donorage                float64
Donorage35                int64
IIIV                      int64
Gendermatch               int64
DonorABO                  int64
RecipientABO             object
RecipientRh              object
ABOmatch                 object
CMVstatus                object
DonorCMV                 object
RecipientCMV             object
Disease                  object
Riskgroup                 int64
Txpostrelapse             int64
Diseasegroup              int64
HLAmatch                  int64
HLAmismatch               int64
Antigen                  object
Alel                     object
HLAgrI                    int64
Recipientage            float64
Recipientage10            int64
Recipientageint           int64
Relapse                   int64
aGvHDIIIIV                int64
extcGvHD                 object
CD34kgx10d6             float64
CD3dCD34                 object
CD3dkgx10d8              object
Rbodymas

In [ ]:
dt.dropna()
#used to remove rows with missing or null values from a pandas DataFrame object dt.

,Recipientgender,Stemcellsource,Donorage,Donorage35,IIIV,Gendermatch,DonorABO,RecipientABO,RecipientRh,ABOmatch,...,extcGvHD,CD34kgx10d6,CD3dCD34,CD3dkgx10d8,Rbodymass,ANCrecovery,PLTrecovery,time_to_aGvHD_III_IV,survival_time,survival_status
0,1,1,22.830137,0,1,0,1,1,1,0,...,1,7.20,1.33876,5.38,35,19,51,32,999,0
1,1,0,23.342466,0,1,0,-1,-1,1,0,...,1,4.50,11.078295,0.41,20.6,16,37,1000000,163,1
2,1,0,26.394521,0,1,0,-1,-1,1,0,...,1,7.94,19.01323,0.42,23.4,23,20,1000000,435,1
3,0,0,39.684932,1,1,0,1,2,1,1,...,?,4.25,29.481647,0.14,50,23,29,19,53,1
4,0,1,33.358904,0,0,0,1,2,0,1,...,1,51.85,3.972255,13.05,9,14,14,1000000,2043,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,1,1,37.575342,1,1,0,1,1,0,0,...,1,11.08,2.52275,4.39,44,15,22,16,385,1
183,0,1,22.895890,0,0,0,1,0,1,1,...,1,4.64,1.038858,4.47,44.5,12,30,1000000,634,1
184,0,1,27.347945,0,1,0,1,-1,1,1,...,1,7.73,1.635559,4.73,33,16,16,1000000,1895,0
185,1,1,27.780822,0,1,0,1,0,1,1,...,0,15.41,8.07777,1.91,24,13,14,54,382,1


In [ ]:
cor=abs(dt.corr(method="pearson"))
relevant_features = cor[cor<0.5]
list4=relevant_features.keys().to_list()
list4
# identify relevant features from a pandas DataFrame object dt by calculating the correlation coefficients among its columns using the Pearson correlation method.

['Recipientgender',
 'Stemcellsource',
 'Donorage',
 'Donorage35',
 'IIIV',
 'Gendermatch',
 'DonorABO',
 'Riskgroup',
 'Txpostrelapse',
 'Diseasegroup',
 'HLAmatch',
 'HLAmismatch',
 'HLAgrI',
 'Recipientage',
 'Recipientage10',
 'Recipientageint',
 'Relapse',
 'aGvHDIIIIV',
 'CD34kgx10d6',
 'ANCrecovery',
 'PLTrecovery',
 'time_to_aGvHD_III_IV',
 'survival_time',
 'survival_status']

In [ ]:
coloums=['Recipientgender','Stemcellsource','Donorage','Donorage35','IIIV','Gendermatch','DonorABO','Riskgroup','Txpostrelapse','Diseasegroup','HLAmatch','HLAmismatch','HLAgrI','Recipientage','Recipientage10','Recipientageint','Relapse','aGvHDIIIIV','CD34kgx10d6','ANCrecovery','PLTrecovery','time_to_aGvHD_III_IV','survival_time',]
x=dt[coloums]
y=dt["survival_status"]


In [ ]:
from sklearn.preprocessing import StandardScaler


In [ ]:
scaler=StandardScaler()
x=scaler.fit_transform(x)
# standardize features in a dataset by removing the mean and scaling to unit variance
#standardize our data before training machine learning models.
# method computes the mean and standard deviation of the data and then applies the transformation to the data.

In [ ]:
x

,Recipientgender,Stemcellsource,Donorage,Donorage35,IIIV,Gendermatch,DonorABO,Riskgroup,Txpostrelapse,Diseasegroup,...,Recipientage,Recipientage10,Recipientageint,Relapse,aGvHDIIIIV,CD34kgx10d6,ANCrecovery,PLTrecovery,time_to_aGvHD_III_IV,survival_time
0,1,1,22.830137,0,1,0,1,1,0,1,...,9.6,0,1,0,0,7.20,19,51,32,999
1,1,0,23.342466,0,1,0,-1,0,0,1,...,4.0,0,0,1,1,4.50,16,37,1000000,163
2,1,0,26.394521,0,1,0,-1,0,0,1,...,6.6,0,1,1,1,7.94,23,20,1000000,435
3,0,0,39.684932,1,1,0,1,0,0,1,...,18.1,1,2,0,0,4.25,23,29,19,53
4,0,1,33.358904,0,0,0,1,1,0,1,...,1.3,0,0,0,1,51.85,14,14,1000000,2043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,1,1,37.575342,1,1,0,1,1,1,1,...,12.9,1,2,1,0,11.08,15,22,16,385
183,0,1,22.895890,0,0,0,1,1,0,1,...,13.9,1,2,0,1,4.64,12,30,1000000,634
184,0,1,27.347945,0,1,0,1,0,0,0,...,10.4,1,2,0,1,7.73,16,16,1000000,1895
185,1,1,27.780822,0,1,0,1,0,0,1,...,8.0,0,1,0,0,15.41,13,14,54,382


In [ ]:
df = pd.DataFrame(x)
df.to_csv("processed_data.csv")
#save the contents of a pandas DataFrame object df into a CSV  file named "processed_data.csv".

In [ ]:
y.value_counts()

0    102
1     85
Name: survival_status, dtype: int64

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE()
x,y=sm.fit_resample(x, y)
y.value_counts()
# SMOTE algorithm to balance the distribution of classes in the dataset represented by x and y

0    102
1    102
Name: survival_status, dtype: int64

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(x, y, random_state=1,train_size=0.8)
#splitng tesing and training 
#split the dataset represented by x and y into training and validation sets for machine learning modeling purposes

In [ ]:
rnn = Sequential()#model creating 

In [ ]:
rnn.add(LSTM(units = 45, return_sequences = True, input_shape = (train_X.shape[1], 1)))
#code is used to add an LSTM layer with 45 memory cells to an RNN model for sequence prediction.

In [ ]:
rnn.add(Dropout(0.2))

In [ ]:
rnn.add(LSTM(units = 45, return_sequences = True))

rnn.add(Dropout(0.2))

rnn.add(LSTM(units = 45, return_sequences = True))

rnn.add(Dropout(0.2))

rnn.add(LSTM(units = 45))

rnn.add(Dropout(0.2))
# build an RNN model for sequence prediction with an architecture consisting of three LSTM layers and three Dropout layers.

In [ ]:
rnn.add(Dense(units = 1))
# add a fully connected layer with one unit to an RNN model for sequence prediction

In [ ]:
rnn.compile(optimizer = 'adam', loss = 'mean_squared_error')#modle comilation 
#After compiling the RNN model, it can be trained and evaluated on a dataset containing sequential data. 

In [ ]:
rnn.fit(train_X,train_y, epochs = 100, batch_size = 32)
#training of data
# train a Recurrent Neural Network (RNN) model for sequence prediction using the fit() function.

Epoch 1/100
6/6 [==============================] - 9s 100ms/step - loss: 0.3825
Epoch 2/100
6/6 [==============================] - 1s 102ms/step - loss: 0.2841
Epoch 3/100
6/6 [==============================] - 1s 102ms/step - loss: 0.2499
Epoch 4/100
6/6 [==============================] - 1s 100ms/step - loss: 0.2574
Epoch 5/100
6/6 [==============================] - 0s 61ms/step - loss: 0.2578
Epoch 6/100
6/6 [==============================] - 0s 60ms/step - loss: 0.2425
Epoch 7/100
6/6 [==============================] - 0s 60ms/step - loss: 0.2581
Epoch 8/100
6/6 [==============================] - 0s 57ms/step - loss: 0.2492
Epoch 9/100
6/6 [==============================] - 0s 57ms/step - loss: 0.2611
Epoch 10/100
6/6 [==============================] - 0s 61ms/step - loss: 0.2457
Epoch 11/100
6/6 [==============================] - 0s 57ms/step - loss: 0.2535
Epoch 12/100
6/6 [==============================] - 0s 57ms/step - loss: 0.2567
Epoch 13/100
6/6 [===========================

In [ ]:
#preiction of data
predictions = rnn.predict(val_X)

2/2 [==============================] - 2s 12ms/step


In [ ]:
from sklearn.metrics import mean_squared_error as MSE
# imports the mean squared error (MSE)

In [ ]:
#checking mean square error
mse_rnn = MSE(val_y,predictions)

In [ ]:
mse_rnn

0.21614256824575467

In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics import classification_report

In [ ]:
#printing evaution metrics
print('\nRNN')
print('Accuracy Score:',metrics.accuracy_score(val_y,predictions.round())*100,'%',sep='')
print('Confusion Matrix:',metrics.confusion_matrix(val_y,predictions.round()),sep='\n')
print(classification_report(val_y,predictions.round()))


RNN
Accuracy Score:63.41463414634146%
Confusion Matrix:
[[12  8]
 [ 7 14]]
              precision    recall  f1-score   support

           0       0.63      0.60      0.62        20
           1       0.64      0.67      0.65        21

    accuracy                           0.63        41
   macro avg       0.63      0.63      0.63        41
weighted avg       0.63      0.63      0.63        41



In [ ]:
pip install pyGRNN --ignore-installed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 KB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 KB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.7/299.7 KB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
from pyGRNN import GRNN

In [ ]:
#creating grnn model
AGRNN = GRNN(calibration = 'gradient_search')

In [ ]:
AGRNN.fit(train_X,train_y.ravel())
#train the model on the provided input data.

GRNN(bnds=((0, None), (0, None), (0, None), (0, None), (0, None), (0, None),
           (0, None), (0, None), (0, None), (0, None), (0, None), (0, None),
           (0, None), (0, None), (0, None), (0, None), (0, None), (0, None),
           (0, None), (0, None), (0, None), (0, None), (0, None)),
     calibration='gradient_search',
     sigma=array([0.36929906, 0.40038312, 0.34743781, 0.38676188, 0.21129164,
       0.18131903, 0.43356765, 0.43551845, 0.39793242, 0.78659378,
       0.44099319, 0.16675527, 0.46733534, 0.24807203, 0.18652415,
       0.17970581, 0.20002918, 0.47091561, 0.35144691, 0.4       ,
       0.39999936, 0.4682308 , 0.02198014]))

In [ ]:
sigma = AGRNN.sigma

In [ ]:
#predicting data
prediction = AGRNN.predict(val_X)

In [ ]:
mse_AGRNN = MSE(val_y,predictions)
mse_AGRNN

0.21614256824575467

In [ ]:
#printing evatuation metrics
print('\nGRNN')
print('Accuracy Score:',metrics.accuracy_score(val_y,prediction.round())*100,'%',sep='')
print('Confusion Matrix:',metrics.confusion_matrix(val_y,prediction.round()),sep='\n')
print(classification_report(val_y,prediction.round()))


GRNN
Accuracy Score:95.1219512195122%
Confusion Matrix:
[[20  0]
 [ 2 19]]
              precision    recall  f1-score   support

           0       0.91      1.00      0.95        20
           1       1.00      0.90      0.95        21

    accuracy                           0.95        41
   macro avg       0.95      0.95      0.95        41
weighted avg       0.96      0.95      0.95        41



In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

In [ ]:

#creating ann model
ann = tf.keras.models.Sequential()

In [ ]:
ann.add(tf.keras.layers.Dense(units=6,activation="relu"))
# provided, ann is an instance of a sequential model in Keras, which is a high-level neural networks API provided by TensorFlow. The add method is used to add layers to the model.

In [ ]:
ann.add(tf.keras.layers.Dense(units=6,activation="relu"))


In [ ]:
ann.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))

In [ ]:
ann.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

In [ ]:
ann.fit(train_X,train_y,batch_size=32,epochs = 75)#train the data

Epoch 1/75
6/6 [==============================] - 1s 6ms/step - loss: 0.6661 - accuracy: 0.5521
Epoch 2/75
6/6 [==============================] - 0s 4ms/step - loss: 0.6572 - accuracy: 0.5583
Epoch 3/75
6/6 [==============================] - 0s 4ms/step - loss: 0.6495 - accuracy: 0.5644
Epoch 4/75
6/6 [==============================] - 0s 5ms/step - loss: 0.6419 - accuracy: 0.5706
Epoch 5/75
6/6 [==============================] - 0s 4ms/step - loss: 0.6346 - accuracy: 0.5767
Epoch 6/75
6/6 [==============================] - 0s 4ms/step - loss: 0.6284 - accuracy: 0.5767
Epoch 7/75
6/6 [==============================] - 0s 4ms/step - loss: 0.6232 - accuracy: 0.5767
Epoch 8/75
6/6 [==============================] - 0s 4ms/step - loss: 0.6172 - accuracy: 0.5767
Epoch 9/75
6/6 [==============================] - 0s 5ms/step - loss: 0.6121 - accuracy: 0.5767
Epoch 10/75
6/6 [==============================] - 0s 5ms/step - loss: 0.6065 - accuracy: 0.5828
Epoch 11/75
6/6 [======================

In [ ]:
predict = ann.predict(val_X)

2/2 [==============================] - 0s 7ms/step


In [ ]:
mse_ann = MSE(val_y,predict)
mse_ann

0.12413925540889706

In [ ]:
#printing evaution metrics
print('\nANN')
print('Accuracy Score:',metrics.accuracy_score(val_y,predict.round())*100,'%',sep='')
print('Confusion Matrix:',metrics.confusion_matrix(val_y,predict.round()),sep='\n')
print(classification_report(val_y,predict.round()))


ANN
Accuracy Score:90.2439024390244%
Confusion Matrix:
[[19  1]
 [ 3 18]]
              precision    recall  f1-score   support

           0       0.86      0.95      0.90        20
           1       0.95      0.86      0.90        21

    accuracy                           0.90        41
   macro avg       0.91      0.90      0.90        41
weighted avg       0.91      0.90      0.90        41

